<h1>Optimisation Assignment aka "build it from scratch"</h1>

In [6]:
import numpy as np
import math
import scipy
import random
import pandas as pd
from math import exp

<h2>Question 1: Write a code to find the root of the function, that is, find x where the value of the following function is 0.</h2>
Take the equation 𝐹(𝑥)=𝑥−sin(𝑥).

Note that 𝐹′(𝑥)=1−cos(𝑥). Apply Newton's method with 𝑥0=1 (initial point) and $\epsilon=10^{-5}$ (error should be smaller than this, that is F(x) < $\epsilon$)

In [96]:
def func(x):
    return x - np.sin(x), 1-np.cos(x)

def newtonEquation(x0, eps, maxiters = 100): 
    k = 0
    x = x0
    f, g = func(x)
    while np.abs(f) > eps and k <= maxiters:
        print(f"Iteration {k}: x = {round(x,4)} f(x) = {round(f,5)} g = {round(g, 5)}")
        k += 1
        if g == 0.0:
            g = g+0.01
            return "Solution reached as f' is close to 0"
        try:    
            x = x - f / g
        except:
            message = f'problems at iteration {k}'
            return x, message
        f, g = func(x)
    if np.abs(f) <= eps:
        return round(x, 4), f'Precision has been reached: {round(np.abs(f),6)} <= {eps}'
    else:
        return None, f'Maximum number of iterations reached: {maxiters}'


In [97]:
# Execution
x0  = 1
eps = 1.0e-5
x, message = newtonEquation(x0, eps)

print(f'Solution: {x} \nReason: {message}')

Iteration 0: x = 1 f(x) = 0.15853 g = 0.4597
Iteration 1: x = 0.6551 f(x) = 0.04587 g = 0.20704
Iteration 2: x = 0.4336 f(x) = 0.01346 g = 0.09254
Iteration 3: x = 0.2881 f(x) = 0.00397 g = 0.04123
Iteration 4: x = 0.1918 f(x) = 0.00117 g = 0.01834
Iteration 5: x = 0.1278 f(x) = 0.00035 g = 0.00816
Iteration 6: x = 0.0852 f(x) = 0.0001 g = 0.00363
Iteration 7: x = 0.0568 f(x) = 3e-05 g = 0.00161
Solution: 0.0379 
Reason: Precision has been reached: 9e-06 <= 1e-05


<h1>Multi-Variable Optimization</h1>

Consider the function \\[f(x_1,x_2) = 2x_1^3+6x_1x_2^2 -3x_2^3-150x_1. \\]

<h2>Question 2. Write a code to find the optimal point (optimal solutions) for the 2-variable function. The gradient and the hessian is shown below.</h2>

<b>Note</b> that when calculating -g/H in python, you can use a function such that direction = np.linalg.solve(H, -g) 

Then the gradient is \\[ \nabla f(x) = \left( \begin{array}{c} 6x_1^2+6x_2^2-150 \\ 12x_1x_2-9x_2^2 \end{array} \right), \\]
and the Hessian is  \\[\nabla^2 f(x) = \left( \begin{array}{cc} 12x_1 & 12x_2\\12x_2 & 12x_1-18x_2\end{array} \right). \\]

In [86]:
def f2(x):
    x1 = x[0]
    x2 = x[1]
    return 2*x1**3 + 6*x1*x2**2 - 3*x2**3 - 150*x1
def df2(x):
    x1 = x[0]
    x2 = x[1]
    return np.array([6*x1**2 + 6*x2**2 -150, 12*x1*x2 - 9*x2**2])
def ddf2(x):
    x1 = x[0]
    x2 = x[1]
    return np.array([[12*x1, 12*x2],[12*x2, 12*x1 - 18*x2]])

def newtonEquation_2(x0, eps, maxiters = 100): 

    k = 0
    x = x0
    f = f2(x)
    g = df2(x)
    h = ddf2(x)
    
    while np.abs(g.all()) >= eps and k <= maxiters:
        print(np.round(g,4))
        print(f'derivative value = {f2(x)}')
        k += 1
        if g.all() == 0.0:
            g = g + 0.01
            return "Solution reached as f'' is close to 0 "
        try:    
            x = x + np.linalg.solve(h, -g)
        except:
            message = f'problems on iteration {k}'
            return x, message
        g = df2(x)
        h = ddf2(x)
    if np.abs(g.all()) <= eps:
        return x, f'Precision has been reached: {np.abs(g[0])} <= {eps}'
    else:
        return None, f'Maximum number of iterations reached: {maxiters}'

In [87]:
# Execution
x0  = [1, 1]
eps = 1.0e-5
x, message = newtonEquation_2(x0, eps)

print(f'Solution: x1 = {round(x[0],4)}, x2 = {round(x[1],4)} \nDiagnostic: {message}')

[-138    3]
derivative value = -145
[ 448.8333 -207.5833]
derivative value = -379.69907407407425
[ 84.9079 -49.5697]
derivative value = -320.8667876349568
[ 8.5108 -8.42  ]
derivative value = -300.89376961264463
[ 0.2072 -0.3545]
derivative value = -300.00139036701705
[ 0.0003 -0.0006]
derivative value = -300.00000000322336
[ 0. -0.]
derivative value = -300.00000000000006
Solution: x1 = 3.0, x2 = 4.0 
Diagnostic: Precision has been reached: 0.0 <= 1e-05


<h1>Logistic Regression Using Stochastic Gradient Descent</h1>

In a logistic regression model, the prediction is given by

$p(x_i) = \frac{1}{1+exp(-f(x))}$ where $f(x)$ is given by:

$f(x) = \theta_0+\theta_1x_1+\theta_2x_2...$ <br>
where $f(x)$ is a linear function (hence logistic regression is called generalized linear regression)

We minimize the negative of maximum likelihood function. Thus the loss function we want to minimize is:<br>
$minimize: loss(\theta) = -\frac{1}{n}\sum_i\bigg( \big(y_i (ln(p(x_i))\big)+(1-y_i)(1-ln(p(x_i))\bigg)$

The gradient of the cost function (in direction $j$ is given by (where m is a batch size, you can chose any batch sixe): <br>
$\frac{d(loss(\theta_))}{d\theta} = \frac{1}{m}\sum_i \bigg(y_i-p(x_i)\bigg)x_i^j$

Therefore, steps in direction $j$ is given by:<br>
$\theta_j (n+1) = \theta_j (n)-\alpha \frac{1}{m}\sum_i \bigg(y_i-p(x_i)\bigg)x_i^j$

Try deriving this from the equation above. Hint, keep it as $p(x_i)$ form, easier for derivation.

Check this link to study how the gradients are calculated:
https://www.baeldung.com/cs/gradient-descent-logistic-regression

<h2>Question 3: Write a code to find the coefficients for $\theta_1$ and $\theta_2$ (use at least 10 iterations and find the coefficients after 10 iterations)</h2>

In [7]:
# first 2 columns are the predictor variables and 3rd column is the output variable (0 or 1)
# marks in 2 exams and if they got admission or not
# in most machine learning problems, it is better to normalize the data for the training set.

data = np.array([
[34.62365962451697,78.0246928153624,0],
[30.28671076822607,43.89499752400101,0],
[35.84740876993872,72.90219802708364,0],
[60.18259938620976,86.30855209546826,1],
[79.0327360507101,75.3443764369103,1],
[45.08327747668339,56.3163717815305,0],
[61.10666453684766,96.51142588489624,1],
[75.02474556738889,46.55401354116538,1],
[76.09878670226257,87.42056971926803,1],
[84.43281996120035,43.53339331072109,1],
[95.86155507093572,38.22527805795094,0],
[75.01365838958247,30.60326323428011,0],
[82.30705337399482,76.48196330235604,1],
[69.36458875970939,97.71869196188608,1],
[39.53833914367223,76.03681085115882,0],
[53.9710521485623,89.20735013750205,1],
[69.07014406283025,52.74046973016765,1],
[67.94685547711617,46.67857410673128,0],
[70.66150955499435,92.92713789364831,1],
[76.97878372747498,47.57596364975532,1],
[67.37202754570876,42.83843832029179,0],
[89.67677575072079,65.79936592745237,1],
[50.534788289883,48.85581152764205,0],
[34.21206097786789,44.20952859866288,0],
[77.9240914545704,68.9723599933059,1],
[62.27101367004632,69.95445795447587,1],
[80.1901807509566,44.82162893218353,1],
[93.114388797442,38.80067033713209,0],
[61.83020602312595,50.25610789244621,0],
[38.78580379679423,64.99568095539578,0],
[61.379289447425,72.80788731317097,1],
[85.40451939411645,57.05198397627122,1],
[52.10797973193984,63.12762376881715,0],
[52.04540476831827,69.43286012045222,1],
[40.23689373545111,71.16774802184875,0],
[54.63510555424817,52.21388588061123,0],
[33.91550010906887,98.86943574220611,0],
[64.17698887494485,80.90806058670817,1],
[74.78925295941542,41.57341522824434,0],
[34.1836400264419,75.2377203360134,0],
[83.90239366249155,56.30804621605327,1],
[51.54772026906181,46.85629026349976,0],
[94.44336776917852,65.56892160559052,1],
[82.36875375713919,40.61825515970618,0],
[51.04775177128865,45.82270145776001,0],
[62.22267576120188,52.06099194836679,0],
[77.19303492601364,70.45820000180959,1],
[97.77159928000232,86.7278223300282,1],
[62.07306379667647,96.76882412413983,1],
[91.56497449807442,88.69629254546599,1],
[79.94481794066932,74.16311935043758,1],
[99.2725269292572,60.99903099844988,1],
[90.54671411399852,43.39060180650027,1],
[34.52451385320009,60.39634245837173,0],
[50.2864961189907,49.80453881323059,0],
[49.58667721632031,59.80895099453265,0],
[97.64563396007767,68.86157272420604,1],
[32.57720016809309,95.59854761387875,0],
[74.24869136721598,69.82457122657193,1],
[71.79646205863379,78.45356224515052,1],
[75.3956114656803,85.75993667331619,1],
[35.28611281526193,47.02051394723416,0],
[56.25381749711624,39.26147251058019,0],
[30.05882244669796,49.59297386723685,0],
[44.66826172480893,66.45008614558913,0],
[66.56089447242954,41.09209807936973,0],
[40.45755098375164,97.53518548909936,1],
[49.07256321908844,51.88321182073966,0],
[80.27957401466998,92.11606081344084,1],
[66.74671856944039,60.99139402740988,1],
[32.72283304060323,43.30717306430063,0],
[64.0393204150601,78.03168802018232,1],
[72.34649422579923,96.22759296761404,1],
[60.45788573918959,73.09499809758037,1],
[58.84095621726802,75.85844831279042,1],
[99.82785779692128,72.36925193383885,1],
[47.26426910848174,88.47586499559782,1],
[50.45815980285988,75.80985952982456,1],
[60.45555629271532,42.50840943572217,0],
[82.22666157785568,42.71987853716458,0],
[88.9138964166533,69.80378889835472,1],
[94.83450672430196,45.69430680250754,1],
[67.31925746917527,66.58935317747915,1],
[57.23870631569862,59.51428198012956,1],
[80.36675600171273,90.96014789746954,1],
[68.46852178591112,85.59430710452014,1],
[42.0754545384731,78.84478600148043,0],
[75.47770200533905,90.42453899753964,1],
[78.63542434898018,96.64742716885644,1],
[52.34800398794107,60.76950525602592,0],
[94.09433112516793,77.15910509073893,1],
[90.44855097096364,87.50879176484702,1],
[55.48216114069585,35.57070347228866,0],
[74.49269241843041,84.84513684930135,1],
[89.84580670720979,45.35828361091658,1],
[83.48916274498238,48.38028579728175,1],
[42.2617008099817,87.10385094025457,1],
[99.31500880510394,68.77540947206617,1],
[55.34001756003703,64.9319380069486,1],
[74.77589300092767,89.52981289513276,1],
])
data.shape

(100, 3)

In [73]:
def get_yhat(row, coefs):
    yhat = coefs[0]
    for i in range(len(row)-1):
        yhat += coefs[i + 1] * row[i]
    return 1/ (1 + exp(-yhat))

def sgd(data, learn_rate, max_iter, eps):
    coef = [0 for i in range(len(data[0]))]
    k = 0
    sse_list = []
    sse_improv = 1
    while k <= max_iter and np.abs(sse_improv) >= eps:
        sse = 0
        for row in data:
            yhat = get_yhat(row, coef)
            e = row[-1] - yhat
            sse += e**2
            coef[0] = coef[0] + learn_rate * e * yhat * (1 - yhat)
            for i in range(len(row)-1):
                coef[i + 1] = coef[i + 1] + learn_rate * e * yhat * (1 - yhat) * row[i]
        sse_list.append(sse)
        if k>0:
            sse_improv = sse_list[k] - sse_list[k-1]
        print(f'interation = {k},  error = {round(sse,4)}, error_improvement = {np.abs(round(sse_improv, 4))}')
        k += 1
    return coef
 

In [99]:
# Execution
learn_rate = 0.01
max_iter = 100
eps = 1.0e-5
coefs = sgd(data, learn_rate, max_iter, eps)
teta_1 = coefs[1]
teta_2 = coefs[2]
print(f'teta1 = {round(teta_1,4)}, teta2 = {round(teta_2,4)}')

interation = 0,  error = 60.2382, error_improvement = 1
interation = 1,  error = 59.9745, error_improvement = 0.2637
interation = 2,  error = 51.101, error_improvement = 8.8735
interation = 3,  error = 39.9979, error_improvement = 11.1032
interation = 4,  error = 39.9978, error_improvement = 0.0001
interation = 5,  error = 39.9977, error_improvement = 0.0001
interation = 6,  error = 39.9976, error_improvement = 0.0001
interation = 7,  error = 39.9975, error_improvement = 0.0001
interation = 8,  error = 39.9973, error_improvement = 0.0001
interation = 9,  error = 39.9972, error_improvement = 0.0001
interation = 10,  error = 39.997, error_improvement = 0.0002
interation = 11,  error = 39.9969, error_improvement = 0.0002
interation = 12,  error = 39.9967, error_improvement = 0.0002
interation = 13,  error = 39.9964, error_improvement = 0.0002
interation = 14,  error = 39.9962, error_improvement = 0.0003
interation = 15,  error = 39.9959, error_improvement = 0.0003
interation = 16,  error 